In [2]:
import numpy as np
import networkx as nx
import copy
from higherorder.structures.structures import Grid, Graph, Structure
from higherorder.dynamics.model import Model
from higherorder.dynamics.rules import *
from higherorder.utils import *

#t = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
s = np.array([[1, 0, 0], [1, 1, 0], [0, 1, 0]])
b = np.array([[0, 1, 0], [1, 0, 1], [0, 1, 0]])
z = np.array([[1, 0, 0], [1, 1, 0], [0, 1, 1]])
l = np.array([[1, 0, 0, 0], [1, 1, 0, 0], [0, 1, 1, 0], [0, 0, 1, 1]])
xl = {(0,0): 1, (1,0): 1, (1,1): 1, (2,1): 1, (2,2): 1, (3,2): 1, (3,3): 1}
f = np.array([[1, 1, 0, 0], [1, 0, 0, 0], [0, 1, 0, 1], [0, 0, 0, 1]])
g = np.array([[0, 1, 0], [0, 0, 1], [1, 1, 1]])

xl0 = Grid(xl, width=10, height=10)
xl1 = Grid(xl, width=50, height=50)
glider = Grid(g, width=15, height=15)
glider2 = Grid(g, width=15, height=15,)
glider3 = copy.deepcopy(glider2)
m0 = Grid (s)
m0b = Grid (b)
f1 = Grid (f)
f2 = Grid (f, diagonal_neighbours=False)
f3 = Grid (f, periodic_boundary=False,)
m1 = Grid (z)
m2 = Grid (z, periodic_boundary=False)
m3 = Grid (z, diagonal_neighbours=False)
m4 = Grid (z, periodic_boundary=False, diagonal_neighbours=False)
print(len(m1.connections), len(m2.connections), len(m3.connections), len(m4.connections))

G = nx.Graph([(1, 2), (2, 3), (3, 4)])
G.add_node(5)
G_ = G.copy()
G_.add_node("A")
G_.nodes[5]['t_0'] = 1
G_.nodes["A"]['t_0'] = 2
G__ = G_.copy()
G__.nodes[4]['t_1'] = 2
initial_values = {2: 1, "A": 3}
print(list(G.edges()))
print(list(G.nodes()))
g1 = Graph(G)
g2 = Graph(G_)
g3 = Graph(G__, time_step = 1)
g4 = Graph(G_, initial_values=initial_values)
g1.get_entities(), g2.get_entities(), g3.get_entities(), g4.get_entities()

36 20 18 12
[(1, 2), (2, 3), (3, 4)]
[1, 2, 3, 4, 5]


({1: {'t_0': 0}, 2: {'t_0': 0}, 3: {'t_0': 0}, 4: {'t_0': 0}, 5: {'t_0': 0}},
 {1: {'t_0': 0},
  2: {'t_0': 1},
  3: {'t_0': 0},
  4: {'t_0': 0},
  5: {'t_0': 1},
  'A': {'t_0': 3}},
 {1: {'t_1': 0},
  2: {'t_1': 0},
  3: {'t_1': 0},
  4: {'t_1': 2},
  5: {'t_0': 1, 't_1': 0},
  'A': {'t_0': 2, 't_1': 0}},
 {1: {'t_0': 0},
  2: {'t_0': 1},
  3: {'t_0': 0},
  4: {'t_0': 0},
  5: {'t_0': 1},
  'A': {'t_0': 3}})

In [ ]:
#1 blob generating
import random

def _setup_grid(grid:Grid | np.ndarray = None,
                  size:int = 10,
                ) -> Grid | np.ndarray:
    """
    Sets up the grid for random blob generation.
    """
    if isinstance(grid, np.ndarray):
        width = grid.shape[0]
        height = grid.shape[1]
        raise NotImplementedError("Array form not implemented yet")
    elif grid is None:
        grid = Grid(np.zeros((2*size, 2*size)))
    
    return grid

def random_1_blob(grid:Grid | np.ndarray = None,
                  size:int = 10,
                  return_array = False,
                  key = None,
                  seed = 1) -> Grid | np.ndarray:
    """
    Generates a random 1 blob (connected component) with a given size.
    """
    random.seed(seed)
    grid = _setup_grid(grid, size)
    
    #TODO make it possible to start with non-zero grid and generate still
    candidates = [] #TODO check nonzero in key?
    nonzero_cells = []
    for i in range(size):
        if i==0:
            cell = random.choice(list(grid.get_entities().keys()))
        else:
            cell = random.choice(candidates)
        nonzero_cells.append(cell)
        #TODO speed this up by only checking the new cell's neighbours
        candidates = grid.get_entities_neighbours(nonzero_cells, external_only = True,
                                                  duplicate_removal = True)
        #TODO do a different version where duplicates are not removed, and their quantities weight the chance inversely
    
    if not key:
        key = "t_0"
    for cell in nonzero_cells:
        grid.entities[cell][key] = 1
            
    return grid


def random_2_blob(grid:Grid | np.ndarray = None,
                  size:int = 10,
                  distance:int = 3,
                  return_array = False,
                  key = None,
                  seed = 1) -> Grid | np.ndarray:
    """
    Generates a random 2 blob (connected component) with a given size.
    """
    random.seed(seed)
    grid = random_1_blob(grid, size, return_array, key, seed)
    #TODO cut the blob into two (e.g. along a horizontal or vertical line) and shift away one of the parts
    return grid

In [ ]:
height = 20
width = 20

grids = []
for size in range(6,30):
    for i in range(100): #1000
        grid = Grid(np.zeros((height, width)))
        grid = random_1_blob(grid, size=size, seed=size*100+i)
        if not len(grid.get_nonzero_entities(base_name="t_0")) == size:
            print("Fail", len(grid.get_nonzero_entities(base_name="t_0")), size)